In [256]:
import xarray as xr
import os, shutil
import subprocess
import luit as lt
import numpy as np

In [267]:
# User-provided info
topofile = '/glade/p/work/manab/fcast/PNW_route/ancillary_data/Network_Topology_Region17_2016_03_21_mod.nc'
RID = 17003601     #Reach ID
attr = xr.open_dataset('/glade/p/work/manab/fcast/PNW/summa_zLocalAttributes_columbia_gru.nc')

# User-provided info for SUMMA
summaexe = '/glade/p/work/manab/fcast/newsumma/summa/bin/summa.exe'
masterdir = '/glade/p/work/manab/fcast/HHDW1/'
summafilemanname = 'summa_fileManager_new.txt'
pbstemplatename = 'template_pbs.txt'
restartflag = '-r never'    #Options: [y,m,d,e,never]
logdname = 'log'
jobdname = 'joblists'
pbsdname = 'pbsscripts'

joblist = "/glade/p/work/manab/fcast/HHDW1/joblists/job_1"
pbslist = "/glade/p/work/manab/fcast/HHDW1/pbsscripts/pbs.sh"

summafileman = masterdir + summafilemanname
logdir = masterdir + logdname
pbstemplate = masterdir + pbstemplatename

In [268]:
topo = xr.open_dataset(topofile).set_index(sSeg = 'reachID')

rstart = topo.sel(sSeg = RID)['reachStart'].values
rcount = topo.sel(sSeg = RID)['reachCount'].values
rlist = topo['reachList'].values[rstart-1:rstart+rcount-1]

hlist = []
for i in range(len(rlist)):
    hrustart = topo.isel(sSeg = rlist[i]-1)['upHruStart'].values
    hrucount = topo.isel(sSeg = rlist[i]-1)['upHruCount'].values
    hlist.append(topo['hru_id'].values[hrustart-1:hrustart+hrucount-1])

hrumin = min([min(a) for a in hlist])
hrumax = max([max(a) for a in hlist])

In [274]:
#Find SUMMA HRU indices for run command
hruindices = [ ]
for count,value in enumerate(hlist):
    idx1 = np.where(attr['hruId'].values == value[0])  #Serial index for SUMMA command corresponding to HRUId
    idx2 = np.where(attr['hruId'].values == value[1])
    idx1 = np.array(idx1)+1                             #Due to SUMMA using 1-11723 instead of 0-11722
    idx2 = np.array(idx2)+1
    hruindices.append(idx1)
    hruindices.append(idx2)

hruindices = np.array(hruindices).flatten()


#Create SUMMA run commands
runCommandList = [ ]
for counter, value in enumerate(hruindices):
    runCommand = [summaexe, '-h', str(value), restartflag, '-m', 
                          summafileman, '>', os.path.join(logdir, str(value) + '.txt')]
    #runCommand = [summaexe, '-g', str(value[0]), restartflag, '-m', 
    #                      summafilemanname, '>', os.path.join('logdir', str(value[0]) + '_' + str(value[1]) + '.txt')]
    
    runCommand = " ".join(runCommand)   #Concatenates into a one-liner
    runCommandList.append(runCommand)
    
#Create a single job list
with open(joblist, "w") as text_file:
    for item in runCommandList:
        text_file.write("{} \n".format(item))

In [260]:
#PBS Scripts
with open(pbstemplate, "rt") as fin:
    with open(pbslist, "wt") as fout:
        for line in fin:
            fout.write(line.replace('columbiaTest_NUMBER', 'HHDW1').replace('JOBLIST', joblist))

In [262]:
#RUN
subprocess.run(["qsub", pbslist])

CompletedProcess(args=['qsub', '/glade/p/work/manab/fcast/HHDW1/pbsscripts/pbs.sh'], returncode=0)